In [ ]:
!pip install flask_ngrok
!pip install pyngrok
!ngrok authtoken 2PeRoZuDhQBsOnJZ5KAEdQB8jlD_6tg7SwMUAMvzAFYeXqdNZ
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
import pickle
import gdown
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
gdown.download(id = '1liwwhm01z8NZw9vXTUBz0gL_t98Rrjgm')
gdown.download(id = '16cOGEvt7L_a0E6S3uoGYA5Kh0_lOZUY3')
gdown.download(id = '14M5Lf716q5UtuojgJY2zvYRqP92gLeGg')
gdown.download(id = '1vgn9j9wnZLZjX_giwlvcaFT948LfoZWJ')
gdown.download(id = '1i11XE4mctBXo3vduzaepaRriXr7F7qQs')
gdown.download(id = '1QJbmftsUGKGte7MRV0UM0cYLErzdtGEk')

Downloading...
From: https://drive.google.com/uc?id=1liwwhm01z8NZw9vXTUBz0gL_t98Rrjgm
To: /content/NN_model.pkl
100%|██████████| 2.45M/2.45M [00:00<00:00, 137MB/s]
Downloading...
From: https://drive.google.com/uc?id=16cOGEvt7L_a0E6S3uoGYA5Kh0_lOZUY3
To: /content/title_vectorizer.pkl
100%|██████████| 85.7k/85.7k [00:00<00:00, 50.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=14M5Lf716q5UtuojgJY2zvYRqP92gLeGg
To: /content/body_vectorizer.pkl
100%|██████████| 7.00M/7.00M [00:00<00:00, 250MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vgn9j9wnZLZjX_giwlvcaFT948LfoZWJ
To: /content/index.html
100%|██████████| 464/464 [00:00<00:00, 1.60MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i11XE4mctBXo3vduzaepaRriXr7F7qQs
To: /content/result.html
100%|██████████| 323/323 [00:00<00:00, 1.00MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QJbmftsUGKGte7MRV0UM0cYLErzdtGEk
To: /content/columnsdf.csv
100%|██████████| 678k/678k [00:00<00:00, 113MB/s]


'columnsdf.csv'

### lematizer

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  

    lemmatized_sentence = []
    
    for word, tag in nltk_tagged:
        if tag is None:

            lemmatized_sentence.append(word)
        else:
          try:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
          except:
            lemmatized_sentence.append(word)

    return " ".join(lemmatized_sentence)

### stopword remover

In [ ]:
def remove_stopwords(rev):

    review_tokenized = word_tokenize(rev)
    rev_new = " ".join([i for i in review_tokenized  if i not in stop_words])
    return rev_new

# app

In [ ]:
with open('/content/NN_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('/content/title_vectorizer.pkl', 'rb') as f:
    title_vectorizer = pickle.load(f)

with open('/content/body_vectorizer.pkl', 'rb') as f:
    body_vectorizer = pickle.load(f)

In [ ]:
app = Flask('Real_or_Fake_News', template_folder='/content/')
run_with_ngrok(app)

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        title = str(request.form['title'])
        body = str(request.form['body'])
        
        df = pd.DataFrame(columns = ['Title','Body'])
        new_row= {'Title': title,'Body': body}
        df = df.append(new_row, ignore_index=True)

        df['Title'] = [str(i).lower() for i in df['Title']]
        df['Title'] = df['Title'].str.replace("[^a-zA-Z0-9]", " ") 
        df['Body'] = [str(i).lower() for i in df['Body']]
        df['Body'] = df['Body'].str.replace("[^a-zA-Z0-9]", " ") 
        
        df['Title'] = df['Title'].apply(lambda x: lemmatize_sentence(x))
        df['Body'] = df['Body'].apply(lambda x: lemmatize_sentence(x))

        df['Title'] = [remove_stopwords(r) for r in df['Title']]
        df['Body'] = [remove_stopwords(r) for r in df['Body']]

        t = title_vectorizer.fit_transform(df['Title'])
        feature_names = title_vectorizer.get_feature_names_out()
        vecdf = pd.DataFrame(t.toarray(), columns = feature_names)
        df = df.merge(vecdf,how = 'outer',left_index=True,right_index=True)        

        t = body_vectorizer.fit_transform(df['Body'])
        feature_names = body_vectorizer.get_feature_names_out()
        vecdf = pd.DataFrame(t.toarray(), columns = feature_names)
        df = df.merge(vecdf,how = 'outer',left_index=True,right_index=True)
        
        del df['Title']
        del df['Body']

        colsdf = pd.read_csv('/content/columnsdf.csv')
        colsdf.loc[0] = 0
        
        merged_df = colsdf.copy()
        for column in colsdf.columns:
            if column in df.columns:
                merged_df[column] = df[column]
        merged_df

        prediction = model.predict(merged_df.values)
        a = a = ( 1 if prediction[0][0] >= 0.5 else 0)
        
        return render_template('result.html', prediction=a)
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app 'Real_or_Fake_News'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ecef-35-190-135-25.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:51:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:51:57] "GET /favicon.ico HTTP/1.1" 404 -
<ipython-input-17-9ef5780f9399>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-17-9ef5780f9399>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Title'] = df['Title'].str.replace("[^a-zA-Z0-9]", " ")
<ipython-input-17-9ef5780f9399>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Body'] = df['Body'].str.replace("[^a-zA-Z0-9]", " ")


1/1 [==============================] - 0s 58ms/step


INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:52:27] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:52:33] "GET / HTTP/1.1" 200 -
<ipython-input-17-9ef5780f9399>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-17-9ef5780f9399>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Title'] = df['Title'].str.replace("[^a-zA-Z0-9]", " ")
<ipython-input-17-9ef5780f9399>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Body'] = df['Body'].str.replace("[^a-zA-Z0-9]", " ")


1/1 [==============================] - 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:52:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2023 06:52:44] "GET / HTTP/1.1" 200 -
